# Preuve de Concept (POC) : Prédiction via Fonction
Utilisé pour directement tester les fonctionnalités de l'appli, c'est comme ça que j'ai l'habitude de travailler, c'est donc plus facile à "imaginer"

In [ ]:
import joblib
import pandas as pd
import sys
import os

# Chargement du modèle
try:
    model = joblib.load("model.joblib")
    print("Modèle chargé avec succès !")
    
    # Extraction des composants pour l'interprétabilité
    scaler = model.named_steps['scaler']
    classifier = model.named_steps['model'] # Le classifier s'appelle 'model' dans le pipeline
    feature_names = scaler.get_feature_names_out() # permet de savoir quel feature on va charger ensuite, surtout pour l'interface graphique
    
    print(f"Composants extraits : Scaler + Classifier ({len(feature_names)} features)")
    
except FileNotFoundError:
    print("Erreur : Le fichier 'Modele/model.joblib' est introuvable.")
except Exception as e:
    print(f"Erreur extraction structure : {e}")

Modèle chargé avec succès !
Composants extraits : Scaler + Classifier (10 features)


In [ ]:
def explain_prediction(df_input, scaler, classifier):
    try:
        # 1. Préparation des données (même ordre que l'entraînement)
        feature_names = scaler.get_feature_names_out()
        df_input = df_input[feature_names] # Réorganise les colonnes par sécurité (recommendation suite au dernier plantage)
        X_scaled = scaler.transform(df_input)
        
        # 2. Calcul pur : Contribution = Valeur normalisée * Coefficient
        # On prend [0] car on analyse un seul individu à la fois
        contributions = X_scaled[0] * classifier.coef_[0]
        
        # 3. Création du bilan
        analysis = pd.DataFrame({
            'Variable': feature_names,
            'Contribution': contributions
        })

        # --- TOP 3 FACTEURS DE RISQUE (Positifs) ---
        # Plus c'est haut, plus ça pousse vers le départ (1)
        risques = analysis.sort_values(by='Contribution', ascending=False).head(3)
        risques = risques[risques['Contribution'] > 0] # On ne garde que les vrais risques

        # --- TOP 3 POINTS FORTS (Négatifs) ---
        # Plus c'est bas, plus ça retient l'employé (vers 0)
        points_forts = analysis.sort_values(by='Contribution', ascending=True).head(3)
        points_forts = points_forts[points_forts['Contribution'] < 0]

        # --- AFFICHAGE PROPRE ---
        print("\n" + "="*40)
        print("📊 SYNTHÈSE DES CONTRIBUTIONS")
        print("="*40)
        
        print("\n🔴 TOP RISQUES (Poussent au départ) :")
        if risques.empty:
            print("  > Aucun facteur critique identifié.")
        for _, row in risques.iterrows():
            print(f"  • {row['Variable']:<40} : +{row['Contribution']:.3f}")

        print("\n🟢 TOP POINTS FORTS (Retiennent l'employé) :")
        if points_forts.empty:
            print("  > Aucun point fort marquant.")
        for _, row in points_forts.iterrows():
            print(f"  • {row['Variable']:<40} : {row['Contribution']:.3f}")
        print("="*40)

    except Exception as e:
        print(f"⚠️ Erreur d'analyse : {e}")

def predict_employee(age, revenu, distance, sat_env, hp_sup, promo, sat_equilibre, pee, poste_actuel, anciennete, exp_totale):
    """
    Fonction de prédiction avec validation complète (Accumulation d'erreurs).
    """
    print(f"\n--- Analyse Profil : Age={age}, Revenu={revenu}€ ---")
    
    # Liste pour accumuler toutes les erreurs
    errors = []

    # 1. Contrôle des inputs
    if not (18 <= age <= 70):
        errors.append(f"Âge invalide ({age}) : doit être entre 18 et 70.")
        
    if revenu < 0:
        errors.append(f"Revenu invalide ({revenu}) : ne peut pas être négatif.")
        
    if distance < 0:
        errors.append(f"Distance invalide ({distance}) : ne peut pas être négative.")
        
    if sat_env not in [1, 2, 3, 4]:
        errors.append(f"Satisfaction env. invalide ({sat_env}) : doit être 1, 2, 3 ou 4.")
        
    if sat_equilibre not in [1, 2, 3, 4]:
        errors.append(f"Satisfaction équilibre invalide ({sat_equilibre}) : doit être 1, 2, 3 ou 4.")
        
    if hp_sup not in ["Oui", "Non"]:
        errors.append(f"Heures sup invalides ('{hp_sup}') : doit être 'Oui' ou 'Non'.")
        
    if promo < 0:
        errors.append(f"Années promotion invalides ({promo}) : ne peut pas être négatif.")
        
    if pee < 0:
        errors.append(f"PEE invalide ({pee}) : ne peut pas être négatif.")

    # S'il y a des erreurs, on les affiche toutes et on arrête
    if errors:
        print("ECHEC VALIDATION ({} erreurs) :".format(len(errors)))
        for err in errors:
            print(f"  - {err}")
        return None, None

    try:
        # 2. Calculs (Feature Engineering)
        div_anciennete = anciennete if anciennete > 0 else 1
        ratio_stagnation = poste_actuel / div_anciennete
        
        div_exp = exp_totale if exp_totale > 0 else 1
        revenu_par_exp = revenu / div_exp
        
        hp_sup_bin = 1 if hp_sup == "Oui" else 0

        # 3. Création DataFrame (Ordre Strict)
        data = {
            'revenu_mensuel': [float(revenu)],
            'age': [int(age)],
            'distance_domicile_travail': [float(distance)],
            'satisfaction_employee_environnement': [int(sat_env)],
            'heure_supplementaires': [hp_sup_bin],
            'annees_depuis_la_derniere_promotion': [int(promo)],
            'satisfaction_employee_equilibre_pro_perso': [int(sat_equilibre)],
            'nombre_participation_pee': [int(pee)],
            'ratio_stagnation': [ratio_stagnation],
            'revenu_par_annee_exp': [revenu_par_exp]
        }
        df = pd.DataFrame(data)

        # 4. Prédiction
        if hasattr(model, "predict_proba"):
            proba = model.predict_proba(df)[0][1]
            prediction = 1 if proba > 0.5 else 0
            score = proba
        else:
            prediction = model.predict(df)[0]
            score = float(prediction)

        # 5. Retour Résultat
        if prediction == 1:
            print(f"⚠️  RISQUE DÉMISSION (Confiance : {score:.1%})")
        else:
            print(f"✅  Pas de risque (Confiance départ : {score:.1%})")
            
        # 6. Explication
        explain_prediction(df, scaler, classifier)
            
        return int(prediction), float(score)

    except Exception as e:
        print(f"Erreur Système inattendue : {e}")
        return None, None

### Tests Manuels

In [28]:
# Cas 1 : Profil à risque
predict_employee(
    age=25, revenu=2000, distance=25, sat_env=1, hp_sup="Oui", 
    promo=5, sat_equilibre=1, pee=0, poste_actuel=2, anciennete=2, exp_totale=3
)


--- Analyse Profil : Age=25, Revenu=2000€ ---
⚠️  RISQUE DÉMISSION (Confiance : 96.3%)

🔍 DÉTAILS DE L'ANALYSE :

Facteur de risque :
   [heure_supplementaires] : Actuel=Oui -> Recommandé=Non
   [satisfaction_employee_environnement] : Actuel=1.0 -> Recommandé=Viser 4
   [revenu_mensuel] : Actuel=2000.0 -> Recommandé=Augmenter

Axe d'amélioration (Points Forts) :
   [ratio_stagnation] : Actuel=1.0 (Positif)
   [revenu_par_annee_exp] : Actuel=666.6666666666666 (Positif)


(1, 0.9633234314591862)

In [29]:
# Cas 2 : Profil Stable
predict_employee(
    age=45, revenu=8000, distance=5, sat_env=4, hp_sup="Non", 
    promo=1, sat_equilibre=4, pee=1, poste_actuel=10, anciennete=15, exp_totale=20
)


--- Analyse Profil : Age=45, Revenu=8000€ ---
✅  Pas de risque (Confiance départ : 6.9%)

🔍 DÉTAILS DE L'ANALYSE :

Facteur de risque :
   Aucun facteur majeur.

Axe d'amélioration (Points Forts) :
   [satisfaction_employee_environnement] : Actuel=4.0 (Positif)
   [heure_supplementaires] : Actuel=Non (Positif)
   [satisfaction_employee_equilibre_pro_perso] : Actuel=4.0 (Positif)


(0, 0.06912394577136804)

In [30]:
# Cas 3 : Test Validation Complète (TOUT EST FAUX)
# Objectif : Voir si TOUTES les erreurs sont signalées
predict_employee(
    age=15,          # Invalide (<18)
    revenu=-5000,    # Invalide (<0)
    distance=-10,    # Invalide (<0)
    sat_env=10,      # Invalide (>4)
    hp_sup="Peut-être", # Invalide
    promo=-1,        # Invalide
    sat_equilibre=0, # Invalide (<1)
    pee=-5,          # Invalide
    poste_actuel=1, 
    anciennete=1, 
    exp_totale=1
)


--- Analyse Profil : Age=15, Revenu=-5000€ ---
ECHEC VALIDATION (8 erreurs) :
  - Âge invalide (15) : doit être entre 18 et 70.
  - Revenu invalide (-5000) : ne peut pas être négatif.
  - Distance invalide (-10) : ne peut pas être négative.
  - Satisfaction env. invalide (10) : doit être 1, 2, 3 ou 4.
  - Satisfaction équilibre invalide (0) : doit être 1, 2, 3 ou 4.
  - Heures sup invalides ('Peut-être') : doit être 'Oui' ou 'Non'.
  - Années promotion invalides (-1) : ne peut pas être négatif.
  - PEE invalide (-5) : ne peut pas être négatif.


(None, None)